In [8]:
import cv2 as cv
import numpy as np
import os
from commonfunctions import *
import scipy
import random


In [3]:
# get threshold value
def calcThreshold(hist, accHist, iFrom, iTo):
    iFrom, iTo = int(iFrom), int(iTo)
    numOfPixels = accHist[iTo] - (accHist[iFrom - 1] if iFrom > 0 else 0)
    mean = np.sum(
        hist[iFrom:iTo+1] * np.arange(iFrom, iTo+1)
    ) / numOfPixels
    return round(mean)

# np.arange([1 , 2] < 3)


def avgThreshold(hist, accHist, Tinit):
    mean1 = calcThreshold(hist, accHist, 0, Tinit - 1)
    mean2 = calcThreshold(hist, accHist, Tinit, hist.shape[0] - 1)
    newThreshold = round((mean1 + mean2) / 2)
    return newThreshold if Tinit == newThreshold else avgThreshold(hist, accHist, newThreshold)


def getGlobalThreshold(img):
    img2 = np.copy(img)
    hist = np.histogram(img2, bins=np.arange(256))[0]
    accHist = np.cumsum(hist)
    Tinit = calcThreshold(hist, accHist, 0, hist.shape[0] - 1)
    return avgThreshold(hist, accHist, Tinit)


In [116]:
# convert images to binary


def binary_image(file_name):
    img = cv.imread(file_name)
    threshold_val = getGlobalThreshold(img)
    img = cv.cvtColor(img,  cv.COLOR_BGR2GRAY)
    _, threshold = cv.threshold(img, threshold_val, 255, 0)
    return np.array(threshold, dtype=np.float32)


def read_data(base_directory, is_truthy, images):
    for file_name in os.listdir(base_directory):
        img = binary_image(f"{base_directory}/{file_name}")
        images.append([img, is_truthy])


images = []
read_data('./images/Train/D', 1, images)
read_data('./images/Train/A', 0, images)
random.shuffle(images)


def get_data_set_and_result(images):
    result = []
    data_set = []
    for image in images:
        data_set.append(image[0])
        result.append(image[1])
    return np.array(data_set), np.array(result)


def reshape_data_set(data_set):
    a, b, c = data_set.shape
    return data_set.reshape((a, b * c))


data_set, result = get_data_set_and_result(images)
data_set = reshape_data_set(data_set)

# apply thresholding to convert grayscale to binary image


In [126]:
# training
svm = cv.ml.SVM_create()
svm.setType(cv.ml.SVM_C_SVC)
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

svm.train(data_set, cv.ml.ROW_SAMPLE, result)


(0.0, array([[1.]], dtype=float32))


In [115]:
# testing
images = []
read_data('./images/Test/D', 1)


[[array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255., 255.,
        255., 255., 255., 255., 255., 255., 255., 255., 255., 255., 255.,
        255., 255., 255., 255., 255., 255.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255.,
        255., 255., 255., 255., 255., 255., 255., 255., 255.,   0., 255.,
        255., 255., 255., 255., 255., 255.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255.,   0.,   0.,
        255., 255., 255., 255., 255., 255., 255., 255., 255.,   0.,   0.,
        255., 255., 255., 255., 255., 255.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255., 255., 255.,
        255., 255., 255., 255., 255., 255., 255., 255., 255.,   0.,   0.,
        255., 255., 255., 255., 255., 255.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 255., 255.,
        255., 255., 255., 255., 255., 255., 255., 255., 255.,   0.,   0.,
        255., 255., 255., 255., 255., 255.],
       [  0.,   0.,   0.,   0.,  